In [0]:
#Load libraries
import pandas as pd
import os

In [0]:
def get_characters(source,start = 0, end = 5, as_numeric=False):

  '''This is an auxiliary function that is called by function split_columns.
  It is not meant to be used directly by the user.
  
  Given a string of characters, split it from character number *start* to character number *end*-1.
  Some columns in B3's original dataset are monetary quantities given as strings with trailing zeros.
  Hence, for example, 000001234 to indicate R$12,34. Setting as_numeric=True removes the trailing zeros
  and converts the variable to numeric type.'''

  piece = source.str.slice(start, end)
  
  if as_numeric:
    #Convert to number
    piece = pd.to_numeric(piece)
  else:
    #Remove trailing spaces
    piece = piece.str.strip()
  
  return(piece)

In [0]:

def split_columns(file):

  '''This is an auxiliary function that is called by function get_dataset.
  It is not meant to be used directly by the user.

  The original file from B3 is a single column with sequential characters.
  This function splits that column into multiple columns,
  one with each variable.'''
  
  source = pd.Series(file.iloc[:,0])

  output = pd.DataFrame()
  output['date'] = get_characters(source, 2, 10)
  output['BDI'] = get_characters(source, 10, 12)
  output['security'] = get_characters(source, 12, 24)
  output['market_type'] = get_characters(source, 24, 27)
  output['company'] = get_characters(source, 27, 39)
  output['specification'] = get_characters(source, 39, 49)
  output['currency'] = get_characters(source, 52, 56)
  output['open'] = get_characters(source, 56, 69, as_numeric=True)
  output['high'] = get_characters(source, 69, 82, as_numeric=True)
  output['low'] = get_characters(source, 82, 95, as_numeric=True)
  output['average'] = get_characters(source, 95, 108, as_numeric=True)
  output['close'] = get_characters(source, 108, 121, as_numeric=True)
  output['volume'] = get_characters(source, 170, 188, as_numeric=True)

  return(output)

In [0]:
def get_dataset(file_path):

  '''The main function.
  Read the csv file with the raw data from B3 and puts it into tidy format'''

  import pandas as pd

  file = pd.read_csv(file_path)
  file = file.iloc[:(len(file)-1),:] #Remove last row
  file = split_columns(file)
  file['date'] = pd.to_datetime(file['date'], format='%Y%m%d') #Set date to datetime format

  return(file)

In [9]:
#Unzip files (exactly as downloaded from B3)
!unzip COTAHIST_A2020
!unzip COTAHIST_A2019
!unzip COTAHIST_A2018
!unzip COTAHIST_A2017
!unzip COTAHIST_A2016
!unzip COTAHIST_A2015

Archive:  COTAHIST_A2020.ZIP
  inflating: COTAHIST_A2020.TXT      
Archive:  COTAHIST_A2019.ZIP
  inflating: COTAHIST_A2019.TXT      
Archive:  COTAHIST_A2018.ZIP
  inflating: COTAHIST_A2018.TXT      
Archive:  COTAHIST_A2017.ZIP
  inflating: COTAHIST_A2017.TXT      
Archive:  COTAHIST_A2016.ZIP
  inflating: COTAHIST_A2016.TXT      
Archive:  COTAHIST_A2015.ZIP
  inflating: COTAHIST_A2015.TXT      


In [0]:
file1 = get_dataset('COTAHIST_A2020.TXT')
file2 = get_dataset('COTAHIST_A2019.TXT')
file3 = get_dataset('COTAHIST_A2018.TXT')
file4 = get_dataset('COTAHIST_A2017.TXT')
file5 = get_dataset('COTAHIST_A2016.TXT')
file6 = get_dataset('COTAHIST_A2015.TXT')

In [0]:
assert (file1.columns == file2.columns).all()
assert (file1.columns == file3.columns).all()
assert (file1.columns == file4.columns).all()
assert (file1.columns == file5.columns).all()
assert (file1.columns == file6.columns).all()

In [12]:
file = pd.concat([file1, file2,file3, file4, file5, file6])
print(file.shape)

(3037272, 13)


In [0]:
#Save file
file.to_parquet(path='B3.gzip')

In [14]:
#Check if it worked by loading file again and seeing a sample
file_again = pd.read_parquet('B3.gzip')
file_again.sample(5)

,date,BDI,security,market_type,company,specification,currency,open,high,low,average,close,volume
329687,2016-09-14,02,ENGI4,010,ENERGISA,PN N2,R$,413,413,405,406,406,2639400
237025,2015-08-05,96,FJTA2F,020,FORJA TAURUS,DIR PRE N2,R$,1,1,1,1,1,89
49905,2018-02-01,78,GGBRB152,070,GGBRE,PN N1,R$,28,39,28,37,39,22300
683782,2019-11-01,78,VALEL570,070,VALEE,ON NM,R$,12,12,10,10,11,132600
246454,2018-06-07,62,PFRM3T,030,PROFARMA,ON NM,R$,383,384,383,383,384,689798


1